Imports

In [1]:
# Import the required libraries.
import cv2
import os
import glob
from PIL import Image
import random 
from random import randint
import numpy as Numpy
from numpy import asarray
from matplotlib import pyplot as plt
import splitfolders

In [3]:
# read folders
base_dir = "../Material-Recognition-Data/image/"
augmented_dir = "../Material-Recognition-Data/augmented_images/"

Augmenting Method

In [15]:
def fill(img, h, w):
    img = cv2.resize(img, (w, h), cv2.INTER_CUBIC)
    return img 

def zoom(img):
    val = random.uniform(0.5, 1)
    h, w = img.shape[:2]
    h_taken = int(val*h)
    w_taken = int(val*w)
    h_start = randint(0, h-h_taken)
    w_start = randint(0, w-w_taken)
    augmented_img = img[h_start:h_start+h_taken, w_start:w_start+w_taken, :]
    augmented_img = fill(augmented_img, h, w)
    return augmented_img

def vertical_shift(img):
    ratio = random.uniform(-0.7, 0.7)
    h, w = img.shape[:2]
    to_shift = h*ratio
    if ratio > 0: 
        augmented_img = img[:int(h-to_shift), :, :]
    else:
        augmented_img = img[int(-1*to_shift):, :, :]
    augmented_img = fill(augmented_img, h, w)
    return augmented_img

def horizontal_shift(img):
    ratio = random.uniform(-0.7, 0.7)
    h, w = img.shape[:2]
    to_shift = w*ratio
    if ratio > 0:
        augmented_img = img[:, :int(w-to_shift), :]
    else:
        augmented_img = img[:, int(-1*to_shift):, :]
    augmented_img = fill(augmented_img, h, w)
    return augmented_img


def augment_images(images_list, class_name, total_augment):
    class_dir = augmented_dir + class_name + "/"
    images_per_class = total_augment 

    if not os.path.isdir(class_dir):
        os.makedirs(class_dir)

    original_count = len(images_list)
    no_required_images = images_per_class - original_count
    print("Augmenting " + str(no_required_images) + " images for this category ...")

    # augment images
    while no_required_images > 0: 
        copy_img = images_list[no_required_images % original_count]
        ran_choice = randint(1, 5)

        # apply diff transformations based on random selection
        if ran_choice == 1:
            # rotate counter clockwise
            augmented_img = cv2.flip(copy_img, 0)
        elif ran_choice == 2:
            # rotate clockwise
            augmented_img = cv2.flip(copy_img, 1)
        elif ran_choice == 3:
            # random zoom on image
           augmented_img = zoom(copy_img)
        elif ran_choice == 4:
            # vertical shift image
            augmented_img = vertical_shift(copy_img)
        elif ran_choice == 5:
            # horizontal shift image
            augmented_img = horizontal_shift(copy_img)

        # resize image 
        resized_img = cv2.resize(augmented_img, (0, 0), fx=0.5, fy=0.5)
        # append to existing list
        images_list.append(resized_img)
        # minus required images
        no_required_images -= 1

    # save all images to new dir 
    for idx in range(len(images_list)):
        curr_img = images_list[idx]

        # convert BGR format to RGB format
        curr_img = cv2.cvtColor(curr_img, cv2.COLOR_BGR2RGB)

        # convert array back to img and save
        curr_img = Image.fromarray(curr_img)
        # pad index with 0s 
        img_idx = f'{idx+1:04}'
        img_name = class_name + "_" + img_idx + ".png"
        curr_img.save(class_dir + img_name)


Read images and augment

In [16]:
image_folders = glob.glob(base_dir + "*")
image_folders = sorted(image_folders)

for folder in range(len(image_folders)):
    # get class name
    class_name = os.path.basename(image_folders[folder])
    # get all paths of images in class
    class_path_list = glob.glob(base_dir + class_name + "/*")

    # get list of images in class 
    images_list = [] 
    for idx in range(len(class_path_list)):
        if class_path_list[idx].endswith(".jpg"):
            curr_img = cv2.imread(class_path_list[idx])
            # convert image into numpy array
            if type(curr_img).__module__ != "numpy":
                curr_img = Numpy.asarray(curr_img)

            images_list.append(curr_img)

    print("Current class that is being processed: " + class_name)
    print("Current amount of images in class: " + str(len(images_list)))
    # augment
    augment_images(images_list, class_name, 1000)

    print("-------------------------------------------------------")

Current class that is being processed: fabric
Current amount of images in class: 100
Augmenting 900 images for this category ...
-------------------------------------------------------
Current class that is being processed: foliage
Current amount of images in class: 100
Augmenting 900 images for this category ...
-------------------------------------------------------
Current class that is being processed: glass
Current amount of images in class: 100
Augmenting 900 images for this category ...
-------------------------------------------------------
Current class that is being processed: leather
Current amount of images in class: 100
Augmenting 900 images for this category ...
-------------------------------------------------------
Current class that is being processed: metal
Current amount of images in class: 100
Augmenting 900 images for this category ...
-------------------------------------------------------
Current class that is being processed: paper
Current amount of images in cl

Train Test Split

In [4]:
# create split data folder
split_dir = "../Material-Recognition-Data/split_data/"
if not os.path.isdir(split_dir):
        os.makedirs(split_dir)

# split data
splitfolders.ratio(augmented_dir, output=split_dir, seed=1337, ratio=(0.8, 0.1, 0.1))

Copying files: 10000 files [04:09, 40.15 files/s]


Mask Images

In [ ]:
width = 256
height = 192

In [ ]:
def read_mask():
    images = []
    labels = []
    imgLabelList = [[]]
    

    pass 

In [ ]:
def mask_img(img):
    blur_img = cv2.GaussianBlur(img, (5, 5), 0)
    hsv_img = cv2.cvtColor(blur_img, cv2.COLOR_BGR2HSV)
    lower_green = (25, 40, 50)
    upper_green = (75, 255, 255)
    mask = cv2.inRange(hsv_img, lower_green, upper_green)
    kernel = cv2.getStructuringElement(cv2.MORPH_ELLIPSE, (11, 11))
    mask = cv2.morphologyEx(mask, cv2.MORPH_CLOSE, kernel)

    b_mask = mask > 0 

    processed_img = Numpy.zeros_like(img, Numpy.uint8)
    processed_img[b_mask] = img[b_mask]

    plt.subplot(2, 3, 1); plt.imshow(img)  # Show the original image
    plt.subplot(2, 3, 2); plt.imshow(blur_img)  # Blur image
    plt.subplot(2, 3, 3); plt.imshow(hsv_img)  # HSV image
    plt.subplot(2, 3, 4); plt.imshow(mask)  # Mask
    plt.subplot(2, 3, 5); plt.imshow(b_mask)  # Boolean mask
    plt.subplot(2, 3, 6); plt.imshow(processed_img)  # Image without background
    
